INPUT -> [CONV -> RELU]*3 -> FC -> RELU -> DROPOUT > SOFTMAX<br>
CONV 필터 크기는 3x3 이고, Stride 는 2x2 이고, 3층이며 필터 수는 각각 32개, 64개이다.<br>
POOLING은 사용하지 않는다

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160704)
tf.set_random_seed(20160704)


[CNN-02] MNIST 데이터 세트를 준비한다.

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from t


[CNN-03] 첫 번째 단계의 합성곱 필터와 풀링 계층을 정의한다.

In [0]:
num_filters1 = 32

x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1,28,28,1])

W_conv1 = tf.Variable(tf.truncated_normal([3,3,1,num_filters1],
                                          stddev=0.1))
h_conv1 = tf.nn.conv2d(x_image, W_conv1,
                       strides=[1,2,2,1], padding='SAME')

b_conv1 = tf.Variable(tf.constant(0.1, shape=[num_filters1]))
h_conv1_cutoff = tf.nn.relu(h_conv1 + b_conv1)


[CNN-04] 두 번째 단계의 합성곱 필터와 풀링 계층을 정의한다.

In [0]:
num_filters2 = 64

W_conv2 = tf.Variable(
            tf.truncated_normal([3,3,num_filters1,num_filters2],
                                stddev=0.1))
h_conv2 = tf.nn.conv2d(h_conv1_cutoff, W_conv2,
                       strides=[1,2,2,1], padding='SAME')

b_conv2 = tf.Variable(tf.constant(0.1, shape=[num_filters2]))
h_conv2_cutoff = tf.nn.relu(h_conv2 + b_conv2)

## CONV, POOL 하나씩 추가해준다

In [0]:
num_filters3 = 128

W_conv3 = tf.Variable(
            tf.truncated_normal([3,3,num_filters2,num_filters3],
                                stddev=0.1))
h_conv3 = tf.nn.conv2d(h_conv2_cutoff, W_conv3,
                       strides=[1,2,2,1], padding='SAME')

b_conv3 = tf.Variable(tf.constant(0.1, shape=[num_filters3]))
h_conv3_cutoff = tf.nn.relu(h_conv3 + b_conv3)


[CNN-05] 전 결합층, 드롭아웃 계층, 소프트맥스 함수를 정의한다.

In [6]:
h_conv3_cutoff_flat = tf.reshape(h_conv3_cutoff, [-1, 4*4*num_filters3])

num_units1 = 4*4*num_filters3
num_units2 = 1024

w3 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
b3 = tf.Variable(tf.constant(0.1, shape=[num_units2]))
hidden3 = tf.nn.relu(tf.matmul(h_conv3_cutoff_flat, w3) + b3)

keep_prob = tf.placeholder(tf.float32)
hidden3_drop = tf.nn.dropout(hidden3, keep_prob)

w0 = tf.Variable(tf.zeros([num_units2, 10]))
b0 = tf.Variable(tf.zeros([10]))
p = tf.nn.softmax(tf.matmul(hidden3_drop, w0) + b0)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



[CNN-06] 오차 함수 loss, 트레이닝 알고리즘 train_step, 정답률 accuracy을 정의한다.

In [0]:
t = tf.placeholder(tf.float32, [None, 10])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


[CNN-07] 세션을 준비하고 Variable을 초기화한다.

In [8]:
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
# saver = tf.train.Saver() saver 필요 없으므로 주석처리

Instructions for updating:
Use `tf.global_variables_initializer` instead.


[CNN-08] 파라미터 최적화를 20000회 반복한다.

최종적으로 테스트 세트에 대해 약 99%의 정답률을 얻을 수 있다.

In [9]:
i = 0
for _ in range(20000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(50)
    sess.run(train_step,
             feed_dict={x:batch_xs, t:batch_ts, keep_prob:0.5})
    if i % 500 == 0:
        loss_vals, acc_vals = [], []
        for c in range(4):
            start = (int)(len(mnist.test.labels) / 4 * c)
            end = (int)(len(mnist.test.labels) / 4 * (c+1))
            loss_val, acc_val = sess.run([loss, accuracy],
                feed_dict={x:mnist.test.images[start:end],
                           t:mnist.test.labels[start:end],
                           keep_prob:1.0})
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)
        loss_val = np.sum(loss_vals)
        acc_val = np.mean(acc_vals)
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))
        #saver.save(sess, 'cnn_session', global_step=i) saver 필요 없으므로 주석처리

Step: 500, Loss: 2485.467529, Accuracy: 0.924000
Step: 1000, Loss: 1741.250732, Accuracy: 0.948600
Step: 1500, Loss: 1358.299316, Accuracy: 0.959400
Step: 2000, Loss: 1162.624023, Accuracy: 0.963900
Step: 2500, Loss: 1096.795410, Accuracy: 0.967200
Step: 3000, Loss: 981.356812, Accuracy: 0.968600
Step: 3500, Loss: 834.316772, Accuracy: 0.974100
Step: 4000, Loss: 795.587402, Accuracy: 0.975500
Step: 4500, Loss: 722.895264, Accuracy: 0.977000
Step: 5000, Loss: 706.344238, Accuracy: 0.976500
Step: 5500, Loss: 699.070007, Accuracy: 0.977000
Step: 6000, Loss: 615.580994, Accuracy: 0.980500
Step: 6500, Loss: 598.287354, Accuracy: 0.980600
Step: 7000, Loss: 628.060364, Accuracy: 0.980900
Step: 7500, Loss: 584.958496, Accuracy: 0.981100
Step: 8000, Loss: 603.274231, Accuracy: 0.980900
Step: 8500, Loss: 506.202881, Accuracy: 0.983600
Step: 9000, Loss: 484.459778, Accuracy: 0.984400
Step: 9500, Loss: 552.302979, Accuracy: 0.982300
Step: 10000, Loss: 482.846588, Accuracy: 0.983700
Step: 10500, Lo